# Build a Summary Agent Using Wikipedia Pages

In [33]:
import requests
from requests.exceptions import RequestException

def fetch_url(url: str) -> str:
    """
    Fetch the Markdown content of a web page using the Jina Reader service.

    This function prepends the Jina Reader proxy URL to the provided `url`,
    sends a GET request with a timeout, and decodes the response as UTF-8 text.

    Args:
        url (str): The URL of the page to fetch.

    Returns:
        Optional[str]: The Markdown-formatted content of the page if the request
        succeeds; otherwise, None.

    Raises:
        None: All network or decoding errors are caught and suppressed.
               Logs or error messages could be added as needed.
    """
    jina_reader_base_url = 'https://r.jina.ai/'
    jina_reader_url = jina_reader_base_url + url

    try:
        response = requests.get(jina_reader_url, timeout=10)
        response.raise_for_status()
    except RequestException as e:
        raise ValueError(f"Failed to fetch URL '{url}': {e}") from e

    try:
        return response.content.decode('utf-8')
    except UnicodeDecodeError as e:
        raise ValueError(f"Failed to decode response content for URL '{url}': {e}") from e

In [34]:
from minsearch import AppendableIndex
from typing import List, Dict, Any, Optional

class SearchTools:

    def __init__(self) -> None:
        self.index = AppendableIndex(text_fields=['summary'])

    def search(self, query: str) -> List[Dict[str, Any]]:
        """
        Search the index for documents matching a query string.

        Args:
            query (str): The search query.

        Returns:
            List[Dict[str, Any]]: A list of search result dictionaries.
        """
        return self.index.search(query, num_results=5)

    def save_summary(self, url: str, summary: Optional[str] = None) -> str:
        """
        Save the summary of a url

        Args:
            url (str): link to the web page.
            summary (str): summary of the web page contents

        Returns:
            str: "SUCCESS" upon successful indexing.
        """
        doc = {
            "url": url,
            "summary": summary
        }
        self.index.append(doc)

        return "SUCCESS"

In [35]:
search_tools = SearchTools()

In [36]:
from pydantic_ai.messages import FunctionToolCallEvent

class NamedCallback:

    def __init__(self, agent):
        self.agent_name = agent.name

    async def print_function_calls(self, ctx, event):
        # Detect nested streams
        if hasattr(event, "__aiter__"):
            async for sub in event:
                await self.print_function_calls(ctx, sub)
            return

        if isinstance(event, FunctionToolCallEvent):
            tool_name = event.part.tool_name
            args = event.part.args
            print(f"TOOL CALL ({self.agent_name}): {tool_name}({args})")

    async def __call__(self, ctx, event):
        return await self.print_function_calls(ctx, event)

In [ ]:
from pydantic_ai import Agent

instructions = """
You are a helpful assistant that provides summarized answers to user questions.
Every time the user gives you a url summarize it and save it to the knowledge database.
If the user is just asking you search the knowledge database to answer the question.

You can user provides a url you can fetch data from that web to give a summary
You can save summaries to the knowledge database
You can search the knowledge database
"""

agent_tools = [fetch_url, search_tools.search, search_tools.save_summary]

summarizer = Agent(
    name='summary_agent',
    instructions=instructions,
    tools=agent_tools,
    model='gpt-4o-mini'
)

In [38]:
results = await summarizer.run(
    user_prompt="What is this page about? https://en.wikipedia.org/wiki/Capybara",
    event_stream_handler=NamedCallback(summarizer)
)

TOOL CALL (summary_agent): fetch_url({"url":"https://en.wikipedia.org/wiki/Capybara"})
TOOL CALL (summary_agent): save_summary({"url":"https://en.wikipedia.org/wiki/Capybara","summary":"The capybara (Hydrochoerus hydrochaeris) is the largest living rodent, native to South America. It inhabits savannas and dense forests near water, living in social groups. Known for its barrel-shaped body and webbed feet, capybaras are herbivorous and primarily graze on grasses. They can swim well and are often seen in close proximity to aquatic environments. Capybaras have a stable population but are hunted for meat and leather in some areas. Their docile nature has made them popular in captivity and they are often seen in zoos and parks."})


In [39]:
print(results.output)

The page is about the capybara (Hydrochoerus hydrochaeris), the largest living rodent native to South America. It describes the capybara's habitat, social behavior, physical characteristics, diet, ecological role, and its interactions with humans. Capybaras are social creatures that often live near water in groups, and they are primarily herbivorous, eating grasses and aquatic plants. Although they are stable in population, they are hunted for meat and leather. Their docile nature has made them popular in captivity, leading to their presence in zoos and parks.


In [40]:
results = await summarizer.run(
    user_prompt="What is this page about? https://en.wikipedia.org/wiki/Lesser_capybara",
    event_stream_handler=NamedCallback(summarizer)
)

TOOL CALL (summary_agent): fetch_url({"url":"https://en.wikipedia.org/wiki/Lesser_capybara"})
TOOL CALL (summary_agent): save_summary({"url":"https://en.wikipedia.org/wiki/Lesser_capybara","summary":"The lesser capybara (_Hydrochoerus isthmius_) is a large semi-aquatic rodent native to South America, particularly found in areas close to water. Smaller than the common capybara, it has a dark brown coat and reaches up to 3 feet in length. This herbivorous species primarily eats grasses and aquatic plants, and can weigh up to 62 pounds. They are excellent swimmers, adapting to escape predators, and their mating occurs year-round in water, typically resulting in litters of 3 to 4 young."})


In [43]:
results = await summarizer.run(
    user_prompt="Summarize this https://en.wikipedia.org/wiki/Hydrochoerus",
    event_stream_handler=NamedCallback(summarizer)
)

TOOL CALL (summary_agent): fetch_url({"url":"https://en.wikipedia.org/wiki/Hydrochoerus"})
TOOL CALL (summary_agent): save_summary({"url":"https://en.wikipedia.org/wiki/Hydrochoerus","summary":"The genus Hydrochoerus comprises large rodents from South America, including two living species: the capybara (Hydrochoerus hydrochaeris) and the lesser capybara (Hydrochoerus isthmius), plus three extinct species. Capybaras are the largest rodents, known for their semi-aquatic habitat, social behavior, and diets mainly consisting of grasses. They can weigh up to 65 kg and usually have a gestation period of 130-150 days, commonly birthing four young. Hydrochoerus is part of the family Caviidae and is closely related to the genus Kerodon. Fossils indicate their distribution extended into North America and the Caribbean."})


In [45]:
results = await summarizer.run(
    user_prompt="Neochoerus (extinct genus related to capybaras) — https://en.wikipedia.org/wiki/Neochoerus",
    event_stream_handler=NamedCallback(summarizer)
)

TOOL CALL (summary_agent): fetch_url({"url":"https://en.wikipedia.org/wiki/Neochoerus"})
TOOL CALL (summary_agent): save_summary({"url":"https://en.wikipedia.org/wiki/Neochoerus","summary":"Neochoerus is an extinct genus of rodents closely related to the living capybara. Fossil remains have been found in North America (Mexico and the United States) and South America (Colombia). The genus includes several species such as Neochoerus aesopi and Neochoerus pinckneyi, and it existed during the Pleistocene epoch."})


In [46]:
results = await summarizer.run(
    user_prompt="Caviodon (extinct genus of rodents related to capybaras) — https://en.wikipedia.org/wiki/Caviodon",
    event_stream_handler=NamedCallback(summarizer)
)

TOOL CALL (summary_agent): fetch_url({"url":"https://en.wikipedia.org/wiki/Caviodon"})
TOOL CALL (summary_agent): save_summary({"url":"https://en.wikipedia.org/wiki/Caviodon","summary":"Caviodon is an extinct genus of rodents that lived during the Late Miocene to Late Pliocene (approximately 6.8 to 3.0 million years ago). It is related to modern capybaras and has been found in fossil records from Argentina, Venezuela, and Brazil. Notable species include *C. andalhualensis*, *C. australis*, *C. cuyano*, *C. multiplicatus*, and *C. pozzii*."})


In [47]:
results = await summarizer.run(
    user_prompt="Neochoerus aesopi (extinct species close to capybaras) — https://en.wikipedia.org/wiki/Neochoerus_aesopi",
    event_stream_handler=NamedCallback(summarizer)
)

TOOL CALL (summary_agent): fetch_url({"url":"https://en.wikipedia.org/wiki/Neochoerus_aesopi"})
TOOL CALL (summary_agent): save_summary({"url":"https://en.wikipedia.org/wiki/Neochoerus_aesopi","summary":"Neochoerus aesopi is an extinct species of large rodent that lived in North America during the Pleistocene epoch until about 12,000 years ago. Related to modern capybaras, it weighed approximately 80 kg and its fossils have been found in states such as Florida and South Carolina. The species was first described in 1853 and is part of the Hydrochoerinae subfamily. Unlike capybaras today, N. aesopi lived in North America, migrating from South America during the Great American Interchange."})


In [48]:
results = await summarizer.run(
    user_prompt="What are threats to capybara populations?",
    event_stream_handler=NamedCallback(summarizer)
)

TOOL CALL (summary_agent): search({"query":"threats to capybara populations"})


In [49]:
results.output

'Capybara populations face several threats, primarily due to human activities. Here are the key threats:\n\n1. **Hunting**: Capybaras are hunted for their meat and leather in various regions, which can lead to population declines.\n2. **Habitat Loss**: Development, agriculture, and deforestation are contributing to the loss of their natural habitat, particularly near water bodies where they thrive.\n3. **Human-Wildlife Conflict**: As their habitats overlap with human settlements, conflicts can arise, further threatening their populations.\n\nDespite these threats, capybara populations are currently considered stable in many areas.'